In [16]:
# Importando librería
import pandas as pd

In [2]:
# Revisando la ubicación del archivo
import os

print(os.getcwd())


c:\Users\LENOVO\Documents\02. Profesional\03.Análisis\Funnel de Onboarding Digital\notebooks


In [ ]:
#Importando csv

df = pd.read_csv("../data/raw/onboarding_ab_test_data.csv")

df.head(20)

,user_id,channel,device_os,ab_group,step_reached,dni_scan_duration,onboarding_duration,is_converted
0,480f1364-e765-4ab6-9d17-b5b39cf87c5d,Facebook,Android,Variant,Landing,NaN,23.69,0
1,2b49feb3-276c-4631-bbd5-83fede0b2de1,Referral,Android,Control,Personal_Info,NaN,121.39,0
2,30d64394-c8dd-469e-9202-c815dd993fd7,Organic,Android,Control,Success,23.38,119.39,1
3,ecd28e95-9fbb-46f7-b2f0-a97103d0e4ee,Google Ads,Android,Variant,Personal_Info,NaN,51.55,0
4,36b1cc6d-0cfb-40db-9ba7-798aaeb42ab2,Facebook,Android,Control,Personal_Info,NaN,112.09,0
5,c66ef264-8e96-4b00-ac69-2c6f7590ac81,Facebook,iOS,Variant,Contract,8.50,102.62,0
6,eac4725d-c16c-4ff7-9998-727c414fdf98,Facebook,Android,Variant,Success,17.65,113.28,1
7,323cc92c-d995-4a1c-9671-47daf002c46c,Organic,Android,Control,DNI_Scan,18.79,91.58,0
8,8282ffbe-ce5e-4223-ae06-696db89c9c8c,Google Ads,iOS,Variant,Success,18.64,125.40,1
9,41bde44a-400f-4e1a-876b-efda96301d89,Organic,iOS,Control,Contract,9.90,100.26,0


In [4]:
# Carga y estructura

print(df.shape) #Imprime:(10000, 8)

print(df.dtypes)
# Imprime:
#user_id                 object
#channel                 object
#device_os               object
#ab_group                object
#step_reached            object
#dni_scan_duration      float64
#onboarding_duration    float64
#is_converted             int64
#dtype: object



(10000, 8)
user_id                 object
channel                 object
device_os               object
ab_group                object
step_reached            object
dni_scan_duration      float64
onboarding_duration    float64
is_converted             int64
dtype: object


In [5]:
# Valores nulos

df['dni_scan_duration'].isnull().sum() #Imprime 4,016

#Respondiendo la pregunta:
# existen nulos en el campo 'dni_scan_duration' porque dentro del flujo de onboarding no todos llegan a la etapa de escaner su dni. 
# El cliente puede solo haber llegado hasta la landing o la información personal pero no haber completado los siguientes pasos. Por ello no existe data. 

np.int64(4016)

In [6]:
# Consistencias de tiempo

duracion = df['dni_scan_duration'] > df ['onboarding_duration'] 

df[duracion]

# No existe dato donde el tiempo del esceneo del dni sea mayor a la duración del onboarding.

,user_id,channel,device_os,ab_group,step_reached,dni_scan_duration,onboarding_duration,is_converted


In [7]:
# Mirando el funnel
df['step_reached'].value_counts()

#Imprime:
#step_reached
#Success          5144
#Personal_Info    2546 
#Landing          1470 
#DNI_Scan          592 
#Contract          248
#Name: count, dtype: int64

# Flujo onboarding: Landing - Personal_Info - DNI_Scan - Contract - Success
# Revisando el flujo observo que la converción es alta ya que la mayoría de clientes (51% del total) termina en esa etapa.
# Asimismo, observo que la mayor caída se da en Personal_Info. Se podría revisar el flujo en ese evento para determinar las causas de la fuga.

step_reached
Success          5144
Personal_Info    2546
Landing          1470
DNI_Scan          592
Contract          248
Name: count, dtype: int64

In [8]:
# Comparando la tasa de conversión entre el grupo control y variant
 
df.groupby('ab_group')['is_converted'].mean()

#Imprime:
#ab_group
#Control    0.486139
#Variant    0.542413

# Se concluye que la tasa de conversión promedio Variant es mayor en 5 puntos porcentuales que el grupo Control. 


ab_group
Control    0.486139
Variant    0.542413
Name: is_converted, dtype: float64

In [9]:
# Comparando la tasa de conversión entre los grupos operativos
df.groupby('device_os')['is_converted'].mean()

#Imprime:
#device_os
#Android    0.506062
#iOS        0.533203

#Revisando la información no se encuentro ninguna diferencia que llame la atención. 
#El sistema operativo IOS es mayor que Android en 2 puntos porcentuales.


device_os
Android    0.506062
iOS        0.533203
Name: is_converted, dtype: float64

# Evaluación del evento con mayor caída

In [ ]:
# Definimos el orden lógico de los pasos
pasos = ['Landing', 'Personal_Info', 'DNI_Scan', 'Contract', 'Success']

# Contamos dónde fugó cada usuario
dropouts = df['step_reached'].value_counts()

# Calculamos el funnel acumulativo
funnel = {}
total_users = len(df)

# Lógica: Para estar en 'Landing', debiste llegar al menos a 'Landing'.
# Para estar en 'Personal_Info', debiste llegar a 'Personal_Info' O 'DNI_Scan' O ... 'Success'
for i, paso in enumerate(pasos):
    # Sumamos los conteos de todos los pasos siguientes (incluyendo el actual)
    pasos_posteriores = pasos[i:] 
    count = dropouts[pasos_posteriores].sum()
    funnel[paso] = count

funnel_df = pd.DataFrame(list(funnel.items()), columns=['Paso', 'Usuarios'])
funnel_df['Conversion_Rate_Step'] = funnel_df['Usuarios'].pct_change() + 1 # Conversión respecto al paso anterior
funnel_df['Var'] = funnel_df['Usuarios'].pct_change()
print(funnel_df)

            Paso  Usuarios  Conversion_Rate_Step       Var
0        Landing     10000                   NaN  0.000000
1  Personal_Info      8530              0.853000 -0.147000
2       DNI_Scan      5984              0.701524 -0.298476
3       Contract      5392              0.901070 -0.098930
4        Success      5144              0.954006 -0.045994


## Hallazgos:
Se observa que el mayor punto de fricción del funnel ocurre entre 'Personal_Info' y 'DNI_Scan', donde se pierde aproximadamente un 30% de usuarios. Al ser una cída mayor al resto sugiere un cuello de botella en el proceso de verificación de identidad.

In [11]:
# Compara el TIEMPO de escaneo, no solo si convirtieron
print(df.groupby('device_os')['dni_scan_duration'].describe())

            count       mean       std  min    25%    50%    75%    max
device_os                                                              
Android    4069.0  17.946397  5.022456  2.0  14.59  17.89  21.41  36.80
iOS        1915.0  11.839598  4.985819  2.0   8.16  11.71  15.26  29.68
